In [23]:
# get data from metro across iowa

In [25]:
import pandas as pd
import plotly.express as px
from geopy.distance import geodesic
import csv
from datetime import datetime
from meteostat import Point, Daily, Monthly, Stations

# Import Meteostat library
from meteostat import Stations

# Approximate bounding box for Iowa
MIN_LAT = 40.3754
MAX_LAT = 43.5012
MIN_LON = -96.6395
MAX_LON = -90.1401

# Grid spacing (in miles)
SPACING_MILES = 10

# Function to move north/south or east/west by a set distance
def move(start, bearing_deg, miles):
    return geodesic(miles=miles).destination(start, bearing_deg)

# Generate the grid
points = []
lat = MIN_LAT

while lat <= MAX_LAT:
    lon = MIN_LON
    while lon <= MAX_LON:
        points.append((lat, lon))
        # Move east by SPACING_MILES
        lon = move((lat, lon), 90, SPACING_MILES).longitude
    # Move north by SPACING_MILES
    lat = move((lat, MIN_LON), 0, SPACING_MILES).latitude

# Save to CSV (optional)
with open("iowa_grid_10mi.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Latitude", "Longitude"])
    writer.writerows(points)

print(f"Generated {len(points)} points.")


Generated 749 points.


In [27]:
iowa = pd.DataFrame(points, columns = ['lat','lon'])

px.scatter_mapbox(iowa, lat = 'lat', lon = 'lon', mapbox_style='carto-positron')

In [30]:


# Get nearby weather stations
stations = Stations()
stations = stations.nearby(42.1133, -93.526)
station = stations.fetch(1000)


,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
KAMW0,Ames / Old Orchard Mobile Home Park,US,IA,<NA>,KAMW,41.9921,-93.6218,291.0,America/Chicago,2006-01-01,2025-04-14,2006-01-01,2022-04-24,2006-01-01,2022-01-01,15626.554704
KBNW0,Boone,US,IA,<NA>,KBNW,42.0496,-93.8476,354.0,America/Chicago,2006-01-01,2025-04-14,2006-01-01,2022-04-24,2011-01-01,2022-01-01,27469.906670
KIFA0,Iowa Falls / Racine,US,IA,<NA>,KIFA,42.4708,-93.2700,347.0,America/Chicago,2007-10-16,2025-04-14,2007-10-19,2022-04-24,2014-01-01,2022-01-01,44984.719699
KEBS0,Webster City / Highview,US,IA,<NA>,KEBS,42.4365,-93.8691,342.0,America/Chicago,2006-01-01,2025-04-14,2006-01-01,2022-04-24,2010-01-01,2022-01-01,45699.194537
KIKV0,Ankeny / Oakwood Heights Mobile Home Park,US,IA,<NA>,KIKV,41.6911,-93.5665,277.0,America/Chicago,2006-01-01,2025-04-14,2006-01-01,2022-04-24,2011-01-01,2022-01-01,47065.997856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71447,Melita,CA,MB,71447,CWEI,49.2800,-100.9900,446.0,America/Winnipeg,1990-07-24,2025-02-05,1990-07-25,2024-04-28,1993-01-01,2022-01-01,984392.640014
ZB23L,Gimli,CA,MB,71748,<NA>,50.6300,-97.0500,230.0,America/Winnipeg,2020-01-01,2022-12-14,2006-09-19,2022-12-11,2006-01-01,2022-01-01,984551.874028
KABH0,Ellicott / Truckton,US,CO,<NA>,KABH,38.7500,-104.3000,1840.0,America/Denver,2006-05-01,2025-04-14,2006-05-01,2022-04-24,2007-01-01,2020-01-01,984703.636488


In [32]:
px.scatter_mapbox(station, lat = 'latitude', lon = 'longitude', mapbox_style='carto-positron')

In [ ]:
# Iterate through these stations and get yearly rain fall up into september 


In [44]:
daily_weather = pd.DataFrame()

for i, row in station.iterrows():
    # print(row['latitude'], row['longitude'])

    # Set time period
    start = datetime(2018, 1, 1)
    end = datetime(2025, 12, 31)

    # Create Point for Vancouver, BC
    pull_point = Point(row['latitude'],row['longitude'] )

    # Get daily data for 2018
    data = Daily(pull_point, start, end)
    data = data.fetch()
    data['name'] = row['name']
    data['lat'] = row['latitude']
    data['lon'] = row['longitude']

    daily_weather = pd.concat([daily_weather,data])


In [45]:
daily_weather

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,name,lat,lon
2018-01-01,-25.9,-30.0,-20.0,0.0,NaN,282.0,11.7,NaN,1046.5,NaN,Ames / Old Orchard Mobile Home Park,41.9921,-93.6218
2018-01-02,-19.0,-30.0,-7.2,0.0,NaN,246.0,17.7,NaN,1033.8,NaN,Ames / Old Orchard Mobile Home Park,41.9921,-93.6218
2018-01-03,-14.1,-18.3,-6.7,0.0,NaN,322.0,24.0,NaN,1026.2,NaN,Ames / Old Orchard Mobile Home Park,41.9921,-93.6218
2018-01-04,-16.1,-20.6,-13.3,0.0,NaN,NaN,6.9,NaN,1030.4,NaN,Ames / Old Orchard Mobile Home Park,41.9921,-93.6218
2018-01-05,-15.0,-20.0,-13.3,0.0,NaN,38.0,10.6,NaN,1033.8,NaN,Ames / Old Orchard Mobile Home Park,41.9921,-93.6218
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-12,15.0,7.8,21.1,0.0,NaN,40.0,7.8,NaN,1021.7,NaN,Greenville / Brighton,33.4829,-90.9856
2025-04-13,19.9,12.0,27.0,0.0,NaN,174.0,25.4,NaN,1016.5,NaN,Greenville / Brighton,33.4829,-90.9856
2025-04-14,22.0,17.8,26.5,0.0,NaN,211.0,21.0,NaN,1015.7,NaN,Greenville / Brighton,33.4829,-90.9856
2025-04-15,18.2,11.8,22.0,0.0,NaN,356.0,15.4,NaN,1022.1,NaN,Greenville / Brighton,33.4829,-90.9856
